In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import os
import re
import pickle

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from tqdm import tqdm

import pyLDAvis.gensim

from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel

/tmp2/b05505004/intern/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/tmp2/b05505004/intern/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
path = os.path.join("..","data","./newdata_clean.xlsx")
n_cpu = 15
batch_size = 10000
max_k = 40
max_features = 256  # only consider the top max_features ordered by term frequency across the corpus.
loadpath = "processed_data_lda_wo_html"
#loadpath = "processed_data_not_rmsw"

n_topic = 80

In [3]:
with open(loadpath, "rb") as f:
    output = pickle.load(f)
clean_data = output["clean_data"]
reduced_data = output["reduced_data"]
token_data = output["token_data"]

## LDA Topic Model
如果不移除 stopword 的話效果很差，主題的字都會是 of, for, it...
`dictionary.filter_extremes()` 過濾掉 token 出現次數少於15個句子，或是出現在超過一半的句子中。 
ref: [Topic Modeling and Latent Dirichlet Allocation (LDA) in Python](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [4]:
import gensim
from gensim.models import LdaMulticore
import pprint
print("Data length: {}".format(len(token_data)))
dictionary = gensim.corpora.Dictionary(token_data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in token_data]
print(bow_corpus[4310]) # just print

Data length: 36939
[(1, 1), (4, 1), (5, 1), (10, 1), (11, 3), (12, 1), (24, 1), (31, 3), (33, 1), (47, 1), (53, 1), (63, 1), (76, 2), (80, 2), (108, 1), (112, 1), (114, 2), (140, 1), (158, 1), (174, 1), (203, 1), (220, 1), (227, 1), (232, 1), (234, 1), (236, 3), (246, 2), (252, 1), (265, 2), (266, 1), (268, 2), (297, 1), (303, 2), (320, 1), (333, 1), (334, 1), (386, 1), (402, 1), (408, 1), (410, 1), (431, 1), (441, 2), (442, 2), (544, 1), (574, 1), (627, 1), (659, 1), (719, 1), (764, 1), (808, 1), (913, 2), (963, 1), (974, 1), (980, 1), (1041, 1), (1066, 1), (1069, 2), (1251, 1), (1274, 1), (1339, 2), (1667, 1), (1719, 1), (1812, 1), (2092, 1), (2135, 1)]


### Visualize

In [5]:
def print_topic_example(reduced_data, token_data, model, dictionary, n_topic):
    print("Model:", model)
    total_len = len(reduced_data)
    print("total_len",total_len)

    topic_distribution = [0 for i in range(n_topic)]
    topic_result = []
    example = [[] for i in range(n_topic)]
    for s, token in tqdm(zip(reduced_data, token_data), total=total_len):
        bow_vector = dictionary.doc2bow(token)
        rank = model[bow_vector]
        if len(rank) == 0:
            continue
        index, score = max(rank, key=lambda tup: tup[1])
        #print(index,score)
        topic_distribution[index] += 1
        topic_result.append(index)
        example[index].append((s, score))
        #print(s)
        #print("Score: {}  Topic: {}\n".format(score, model.print_topic(index, 5)))
    print("topic_distribution: {}".format(topic_distribution))
    
    for idx, topic in model.print_topics(-1):
        print('Topic: {} | {} datas\nWord: {}'.format(idx, topic_distribution[idx], topic))
        result = sorted(example[idx], key=lambda tup: -tup[1])[:5]
        for s, score in result:
            print("{} | {}\n".format(s,score))
        print()
        print("====================")
    
    return topic_result

### LDA using BOW

In [6]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=n_topic, id2word=dictionary, passes=2, workers=n_cpu)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"version" + 0.014*"product" + 0.014*"use" + 0.013*"get" + 0.013*"program" + 0.013*"upgrade" + 0.012*"powerdirector" + 0.012*"key" + 0.011*"please" + 0.010*"software"
Topic: 1 
Words: 0.016*"please" + 0.011*"use" + 0.010*"dvd" + 0.009*"get" + 0.009*"window" + 0.009*"week" + 0.008*"help" + 0.008*"video" + 0.008*"like" + 0.008*"photo"
Topic: 2 
Words: 0.012*"clip" + 0.010*"want" + 0.010*"product" + 0.009*"one" + 0.009*"download" + 0.009*"get" + 0.009*"need" + 0.009*"please" + 0.009*"video" + 0.009*"time"
Topic: 3 
Words: 0.013*"use" + 0.012*"video" + 0.012*"work" + 0.011*"download" + 0.010*"screen" + 0.009*"version" + 0.008*"thank" + 0.008*"powerdvd" + 0.008*"purchase" + 0.008*"please"
Topic: 4 
Words: 0.022*"key" + 0.022*"get" + 0.020*"purchase" + 0.018*"version" + 0.016*"upgrade" + 0.015*"product" + 0.013*"download" + 0.012*"powerdirector" + 0.012*"powerdvd" + 0.012*"try"
Topic: 5 
Words: 0.015*"product" + 0.015*"video" + 0.012*"bluray" + 0.010*"please" + 0.009*"d

In [7]:
bow_topic_result = print_topic_example(reduced_data, token_data, lda_model, dictionary, n_topic)

  0%|          | 71/36939 [00:00<00:53, 693.87it/s]

Model: LdaModel(num_terms=3478, num_topics=80, decay=0.5, chunksize=2000)
total_len 36939


100%|██████████| 36939/36939 [00:58<00:00, 631.51it/s]


topic_distribution: [316, 157, 224, 316, 704, 237, 927, 94, 228, 1739, 216, 116, 322, 178, 548, 396, 163, 275, 1434, 434, 608, 320, 321, 147, 243, 193, 2595, 719, 333, 785, 1545, 484, 566, 280, 137, 493, 180, 411, 196, 660, 789, 425, 224, 1761, 363, 146, 273, 885, 156, 121, 397, 357, 268, 321, 212, 440, 122, 417, 468, 250, 744, 319, 464, 447, 325, 195, 432, 448, 254, 165, 282, 211, 181, 952, 194, 501, 216, 1181, 294, 499]
Topic: 0 | 316 datas
Word: 0.018*"version" + 0.014*"product" + 0.014*"use" + 0.013*"get" + 0.013*"program" + 0.013*"upgrade" + 0.012*"powerdirector" + 0.012*"key" + 0.011*"please" + 0.010*"software"
Hi, i have just re-instaleld Powerdirector 16 on my new pc, having used it previously on my old pc.    However, i have noticed that when i try to produce a video with more than one track, the resulting video has "scratches" on it. Every 10 to 20 secnds, at random times, there is a momentary audible scratch, which lasts for about half a second or so.     If i completely del

In [8]:
visual = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(visual)

/tmp2/b05505004/intern/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


### LDA using TF-IDF

In [ ]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
print(corpus_tfidf[4310])

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=n_topic, iterations=100000, id2word=dictionary, passes=2, workers=n_cpu)

for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWord: {}'.format(idx, topic))

# Compute Perplexity
per_score = lda_model_tfidf.log_perplexity(corpus_tfidf)
print('Perplexity: ', per_score)  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=token_data, dictionary=dictionary, coherence='c_v', processes=n_cpu)
coherence_lda = coherence_model_lda.get_coherence() # high is better
print('Coherence Score: ', coherence_lda)

[(1, 0.03167906274638008), (4, 0.05968777406530077), (5, 0.07222822116151202), (10, 0.040443009184883257), (11, 0.15954562838219177), (12, 0.04859589956958277), (24, 0.07193005471181863), (31, 0.1703003208531593), (33, 0.028934071348542197), (47, 0.0488459655168579), (53, 0.053634690951270936), (63, 0.05374512038840211), (76, 0.0865111728116621), (80, 0.12634402810353038), (108, 0.0511671022385774), (112, 0.057175724958900095), (114, 0.06001196434858863), (140, 0.02922130108810969), (158, 0.1303232830278721), (174, 0.03284660598771906), (203, 0.045525216659558415), (220, 0.08044749132910173), (227, 0.10103679008811892), (232, 0.055966722269845476), (234, 0.06566735100029075), (236, 0.16372273628252496), (246, 0.11724486387731832), (252, 0.09770160474729414), (265, 0.18171711222882117), (266, 0.100509915566908), (268, 0.13798893886991023), (297, 0.0995975353008924), (303, 0.14848800981156712), (320, 0.09793814969418894), (333, 0.10952426839118118), (334, 0.11341178827747259), (386, 0.09

In [ ]:
tfidf_topic_result = print_topic_example(reduced_data, token_data, lda_model_tfidf, dictionary, n_topic)

In [ ]:
visual = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
pyLDAvis.display(visual)

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].hist(bow_topic_result, rwidth=0.8)   # , c=label_subset_color
ax[0].set_title('BOW Topic Distribution')
ax[1].hist(tfidf_topic_result, rwidth=0.8)   # , c=label_subset_color
ax[1].set_title('TFIDF Topic Distribution')